# Modeling: Aspect-Based Sentiment Analysis [Simplistic]

**`Goal:`** 

Conduct ABSA using word relatedness and out-of-the-box [ABSA package by ScalaConsultants](https://github.com/ScalaConsultants/Aspect-Based-Sentiment-Analysis). This notebook is meant to serve as a start for tweet aspect annotation by getting as much of the aspects indicated and their corresponding sentiments. 

**Note:** Results will be crosschecked during the annotation phase!

**`Process:`** 
1. List aspects (e.g. speed, price, reliability) determined from earlier data annotation phase
2. Get nouns, adjectives and adverbs from the tweets as these will likely be the parts of speech making meaningful reference to aspects
3. Check if each of the words from step 2 is very similar to any of the aspects (e.g. speed [aspect] and fast [word in tweet]) by computing relatedness score (via word embedding)
4. If relatedness score is past a set thresholdhood, we assume the aspect was referenced in the tweet. Hence, note down that the aspect category was referenced in that given tweet and also note down the word (herein called aspect term) that implied the aspect
6. Conduct ABSA using the ABSA package with the tweet and with the aspect term and note sentiment (positive, negative or neutral) towards the main aspect (price, speed, etc.)
7. If multiple words make reference to a single aspect, find the average of their sentiments and use to assign a single sentiment 

In [1]:
# python -m spacy download en_core_web_md
# python -m spacy download en_core_web_lg

### 1. Library Importation

In [2]:
import pandas as pd
import numpy as np
import re
import aspect_based_sentiment_analysis as absa
import nltk
from nltk import pos_tag, RegexpParser

#Packages for word relatedness computation
import spacy
spacy_nlp = spacy.load('en_core_web_lg')

from itertools import product
from cleantext import clean

2021-11-25 21:59:15.631180: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2. Loading the data

In [3]:
df = pd.read_csv('../data/processed/sample_encoded_and_cleaned_no_punct.csv')

In [4]:
df.head()

,ISP_Name,Time,Text,Source,sentiment,label
0,sprectranet,2020-02-04 18:30:35+00:00,my family used my spectranet and they dont wan...,Twitter for Android,Neutral,1
1,sprectranet,2019-06-19 04:59:49,spectranetng how can i get the freedom mifi in...,Twitter for iPhone,Neutral,1
2,sprectranet,2020-03-30 07:57:38+00:00,drolufunmilayo iconicremi spectranetng,Twitter for iPhone,Neutral,1
3,sprectranet,2020-12-31 21:07:52+00:00,spectranetng your response just proves how hor...,Twitter for Android,Negative,0
4,sprectranet,2020-09-03 23:09:09+00:00,spectranet is just the worse tbh i cant even w...,Twitter for iPhone,Negative,0


In [5]:
df.sentiment.value_counts()

Negative    216
Neutral     131
Positive     30
Name: sentiment, dtype: int64

In [6]:
#Load the model for ABSA modeling
nlp = absa.load()

Some layers from the model checkpoint at absa/classifier-rest-0.2 were not used when initializing BertABSClassifier: ['dropout_379']
- This IS expected if you are initializing BertABSClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertABSClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of BertABSClassifier were not initialized from the model checkpoint at absa/classifier-rest-0.2 and are newly initialized: ['dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#1. List aspects determined during the annotation phase
    #Note: This might not be exhaustive! But it should cover most cases. It is also subjective!
    #Also using synonyms of these words will likely yield different results
aspects = ['price','speed','reliability','coverage', 'customer service', 'trustworthiness']

#2. Pair aspects with their tokenized form to avoid recomputation in the ABSA phase below
aspects_with_token = [] #List to store the pairing

#Iterate through the aspects and compute their word vector using spacy
for aspect in aspects:
    aspects_with_token.append((aspect,spacy_nlp(aspect)))
    
aspects_with_token

[('price', price),
 ('speed', speed),
 ('reliability', reliability),
 ('coverage', coverage),
 ('customer service', customer service),
 ('trustworthiness', trustworthiness)]

In [8]:
#Set to store all seen words
seen_words = set()

#Set to store all aspect implying words found – to avoid recomputing similarity scores
aspect_implying_words_glob = set()

#Dictionary categorizing all aspect-implying words into their relevant aspects
aspects_with_implying_words = {'price':set(),'speed':set(),'reliability':set(),'coverage':set(), 
                               'customer service':set(),'trustworthiness':set()}

#List to store detected aspects and their sentiments
df_list = []

#Similarity threshold
sim_thresh = 0.6

#Chunk tags to match – i.e. parts of speech to extract
CHUNK_TAG = """
MATCH: {<NN>+|<NN.*>+}
{<JJ.*>?}
{<RB.*>?}
"""

#Initialize chunk tag parser
cp = nltk.RegexpParser(CHUNK_TAG)

#Iterate through all the tweets
for tweet in df.Text:
    
    #Set to store the detected aspects at the sentence level
    # detected_aspects = set()
    
    #Dictionary to store the sentiment value for each seen aspect
    sentence_lvl_aspect_sentiment = {'price':[],'speed':[],'reliability':[],'coverage':[], 
                                     'customer service':[], 'trustworthiness':[]}
        
    #Split the tweet into words
    text = tweet.split()

    #Tag the words with their part of speech
    tokens_tag = pos_tag(text)
    
    #Get the words with relevant POS (noun, adverbs, adjectives)
    chunk_result = cp.parse(tokens_tag)
    
    #Extract chunk results from tree into list 
    chunk_items = [list(n) for n in chunk_result if isinstance(n, nltk.tree.Tree)]
    
    #Finally fuse/extract chunked words to get (noun) phrases, nouns, adverbs, adjectives
    #1. List to store the words
    matched_words = []
    
    #2. Iterate through the chunked words lists and get the relevant words
    for item in chunk_items:
        if len(item) > 1:
            full_string = []

            for word in item:
                full_string.append(word[0])

            matched_words.append(' '.join(full_string))

        else:
            matched_words.append(item[0][0])
        
    #Iterate through all the words
    for word_in_focus in matched_words:
        
        #If the word has been seen before
        if word_in_focus in seen_words:
            
            #Check if the word is an aspect-implying word
            if word_in_focus in aspect_implying_words_glob:
                
                #List to store all the aspects found to related to the certain word/token
                aspects_implied = []
            
                #If it is an aspect-implying word, iterate through all the aspects
                for aspect in aspects_with_implying_words.keys():
                    
                    #Check if the word_in_focus was noted as a word implying the aspect
                    if word_in_focus in aspects_with_implying_words[aspect]:
                        
                        #Get all the aspects the word_in_focus implies
                        aspects_implied.append(aspect)
                        
            
            else:
                continue
                    
         
        #If the word hasn't been seen before
        else:
            
            #Mark the word as seen now
            seen_words.add(word_in_focus)
                
            #List to store all the aspects found to related to the certain word/token
            #Ideally a given word won't imply multiple of the aspects as they are fairly independent
            #-but just in case 
            aspects_implied = []

            #Iterate through all the aspects
            for aspect,asp_token in aspects_with_token:

                #Translate word_in_focus to word vector
                spacy_token = spacy_nlp(word_in_focus)

                #Compute the similarity between the two word vectors (i.e. the two words)
                #Round up to 1 d.p.
                similarity_score = round(asp_token.similarity(spacy_token),1)

                #If the max similarity score seen is greater than the threshold
                if similarity_score >= sim_thresh:

                    #Add the word to the set of all aspect-implying words seen
                    aspect_implying_words_glob.add(word_in_focus)

                    #Add the word to the dictionary of the relevant aspect word
                    aspects_with_implying_words[aspect].add(word_in_focus)

                    #Note that the aspect has been found in this particular sentence
                    # detected_aspects.add(aspect)

                    #Add the aspect to the list of aspects that the word_in_focus implies
                    aspects_implied.append(aspect)


                #If the word is not an aspect implying word, continue to next word
                else:

                    continue
                
        #Calculate the sentiment scores for the aspect_implying word in the current sentence
        sentiment = nlp(tweet ,aspects = [word_in_focus])
        sentiment_scores = sentiment.subtasks[word_in_focus].examples[0].scores

        #Note down the scores for all the implied aspects
        for aspect in aspects_implied:
            sentence_lvl_aspect_sentiment[aspect].append(sentiment_scores)
    
    #List to store the detected aspects from the sentence
    detected_aspects = []
    
    #List to store the determined sentiments of the detected aspects
    detected_sentiments = []
    
    #Iterate through all the aspects
    for aspect in sentence_lvl_aspect_sentiment.keys():
        
        #If the aspect was detected in the sentence
        if sentence_lvl_aspect_sentiment[aspect]:
            
            #Record this
            detected_aspects.append(aspect)
            
            #Calculate the average sentiment scores across the different terms
            avg_senti_score = np.array(sentence_lvl_aspect_sentiment[aspect]).mean(axis=0)
            
            #Get the sentiment category (neutral,negative,positive) with the largest probability
            max_idx = np.argmax(avg_senti_score)

            if max_idx == 2:

                detected_sentiments.append("Positive")

            elif max_idx == 1:

                detected_sentiments.append("Negative")

            else:

                detected_sentiments.append("Neutral")
    
    #Add the detected aspects and sentiments from the sentence to the list
    if detected_aspects:
        df_list.append([tweet,detected_aspects,detected_sentiments])
    else:
        df_list.append([tweet,None,None])

            

/Users/koredeakande/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/ipykernel_launcher.py:111: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


In [9]:
absa_df = pd.DataFrame(df_list, 
                       columns=['Tweets','Detected aspects','Corresponding sentiment'])

In [10]:
with pd.option_context('display.max_colwidth', None):
    display(absa_df)

,Tweets,Detected aspects,Corresponding sentiment
0,my family used my spectranet and they dont want to help my ministry now it has finished spectranetng abeg how will i change my password,None,None
1,spectranetng how can i get the freedom mifi in ajah today,None,None
2,drolufunmilayo iconicremi spectranetng,None,None
3,spectranetng your response just proves how horrid your customer service is rather than ask what my issue is to help resolve you render an apology apology accepted now can you actually proffer solution i am really disappointed with your services and attitude to customers,[customer service],[Negative]
4,spectranet is just the worse tbh i cant even watch a 5min video without serious lagging,None,None
...,...,...,...
372,spectranet unlimited value for money,[price],[Positive]
373,from 30th may to date mtn mifi 10k spectranet 10250 smile 24000 mtn mobile data roughly 57k both mtn 14k and smile mifi14500 were bought within the last one month spectranet deceived me isps have frustrated me,None,None
374,spectranetng fritzthejanitor will they help me attend to other issues as well,None,None
375,thefunkydee spectranetng im giving spectranetng a second thoughts with this im this thinking of switching from smile to spectranetng and this doesnt looks good what would you advise me to do,None,None


In [11]:
aspects_with_implying_words

{'price': {'buy spectranetng',
  'price',
  'purchase',
  'value',
  'value spectranet'},
 'speed': {'download speed',
  'fast',
  'internet speed',
  'slow',
  'slower',
  'snail speed',
  'speed',
  'speed abeg',
  'speeds'},
 'reliability': {'network quality', 'reliable', 'usefulness'},
 'coverage': {'coverage', 'insurance claim', 'network coverage'},
 'customer service': {'business',
  'company',
  'customer',
  'customer care',
  'customer care line isnt',
  'customer service',
  'customer service experience',
  'customers',
  'disgusting customer service',
  'ifes business',
  'internet connection',
  'internet service',
  'internet service provider',
  'isp business',
  'network provider',
  'network quality',
  'network reception',
  'network service',
  'provider i',
  'providers',
  'reliable',
  'service',
  'service i',
  'service provider',
  'service subscription failure',
  'services',
  'services i',
  'spectranet ltd internet subscription n18525',
  'teleport service',

In [12]:
absa_df[absa_df['Detected aspects'].notnull()]['Corresponding sentiment'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[Negative]                        40
[Positive]                        18
[Neutral]                         11
[Negative, Negative]               5
[Negative, Neutral]                1
[Positive, Positive, Positive]     1
Name: Corresponding sentiment, dtype: int64

In [13]:
absa_df.to_csv('../data/model-generated/tweet_absa_price_not_cost.csv',index=False)

## Model Evaluation

#### Load the true aspect and their sentiment predictions

In [15]:
true_aspects_df = pd.read_csv("../data/processed/absa_labelled_price_not_cost.csv")
true_aspects_df.head()

,Tweets,Aspects,Sentiment
0,my family used my spectranet and they dont wan...,NaN,NaN
1,spectranetng how can i get the freedom mifi in...,NaN,NaN
2,drolufunmilayo iconicremi spectranetng,NaN,NaN
3,spectranetng your response just proves how hor...,['customer service'],['Negative']
4,spectranet is just the worse tbh i cant even w...,['speed'],['Negative']


In [16]:
true_aspects_df.Aspects[3]

"['customer service']"

#### Reformat from string format to list

In [17]:
#Carry out conversion for Aspects column
true_aspects_df.Aspects = true_aspects_df.Aspects.apply(lambda x: eval(x) if (pd.notnull(x)) else x)

#Carry out conversion for Sentiment column
true_aspects_df.Sentiment = true_aspects_df.Sentiment.apply(lambda x: eval(x) if (pd.notnull(x)) else x)

#### Merge the true predictions and the model's predictions

In [18]:
merged_df = absa_df.copy()
merged_df[['True Aspects','True Sentiment']] = true_aspects_df[['Aspects','Sentiment']]
merged_df.head()

,Tweets,Detected aspects,Corresponding sentiment,True Aspects,True Sentiment
0,my family used my spectranet and they dont wan...,None,None,NaN,NaN
1,spectranetng how can i get the freedom mifi in...,None,None,NaN,NaN
2,drolufunmilayo iconicremi spectranetng,None,None,NaN,NaN
3,spectranetng your response just proves how hor...,[customer service],[Negative],[customer service],[Negative]
4,spectranet is just the worse tbh i cant even w...,None,None,[speed],[Negative]


#### Fill Nones and NaNs with [None]

In [19]:
merged_df = merged_df.apply(lambda s: s.fillna({i: [None] for i in df.index}))
merged_df.head()

,Tweets,Detected aspects,Corresponding sentiment,True Aspects,True Sentiment
0,my family used my spectranet and they dont wan...,[None],[None],[None],[None]
1,spectranetng how can i get the freedom mifi in...,[None],[None],[None],[None]
2,drolufunmilayo iconicremi spectranetng,[None],[None],[None],[None]
3,spectranetng your response just proves how hor...,[customer service],[Negative],[customer service],[Negative]
4,spectranet is just the worse tbh i cant even w...,[None],[None],[speed],[Negative]


### Aspect Extraction Evaluation

In [20]:
def micro_precision_recall_f1_score(true_aspects,aspect_preds):
    
    """
    Function to compute the micro-averaged precision, recall and f1 score based on the model's predicitions
    
    Formulas guided by:
    
        - MICRO-PRECISION:
          Micro-precision on the Peltarion Platform. (2021). Micro-precision on the Peltarion Platform.
          Retrieved from https://peltarion.com/knowledge-center/documentation/evaluation-view/classification-loss-metrics/micro-precision
        
        - MICRO-RECALL:
          Micro-recall on the Peltarion Platform. (2021). Micro-recall on the Peltarion Platform.
          Retrieved from https://peltarion.com/knowledge-center/documentation/evaluation-view/classification-loss-metrics/micro-recall
          
        - MICRO-F1:
          Micro F1-score on the Peltarion Platform. (2021). Micro F1-score on the Peltarion Platform. 
          Retrieved from https://peltarion.com/knowledge-center/documentation/evaluation-view/classification-loss-metrics/micro-f1-score
  
    Inputs:
        - true_aspects (pandas Series): True aspects for each tweet
        - aspect_preds (pandas Series): Model's predicted aspects for each tweet
        
    Outputs:
        - class metrics (dict): Dictionary of class-level metrics: false positive, true positive and precision
        - micro_precision (float): Micro-averaged precision
        - micro_recall (float): Micro-averaged recall
        - micro_f1 (float): Micro-averaged f1 score
    """
    
    #Dictionary to note the number of true positives, false positives and false negatives 
    #for the different classes
    class_metrics = {}
    
    #Iterate through all the aspects
    for aspect in aspects:
        
        #Initialize counters for true positives, false positives and false negatives
        TP, FP, FN, TN = 0, 0, 0, 0
        
        #Iterate through all the tweets
        for idx in range(len(aspect_preds)):
            
            #If the predicted aspect is truly in the tweet
            if (aspect in aspect_preds[idx]) and (aspect in true_aspects[idx]):
                
                #Note a true positive
                TP += 1
            
            #If the aspect is in the tweet but the model does not recognize it
            if (aspect not in aspect_preds[idx]) and (aspect in true_aspects[idx]):
                
                #Note false negative
                FN += 1
                
            #If the predicted aspect is not truly in the tweet
            if (aspect in aspect_preds[idx]) and (aspect not in true_aspects[idx]):

                #Record false positive
                FP += 1
                
            #If the aspect was correctly left out of the tweet
            if (aspect not in aspect_preds[idx]) and (aspect not in true_aspects[idx]):

                #Record false positive
                TN += 1
        
        #Calculate class level precision and recall
        precision = float(TP/(TP+FP))
        recall = float(TP/(TP+FN))
        
        #Calculate class level f1 score
        try:
            f1_score = 2*((precision*recall)/(precision+recall))
        except ZeroDivisionError:
            f1_score = 0

        #Note down the final class-level metrics
        class_metrics[aspect] = {'TP':TP, 'FP':FP, 
                                 'FN': FN, 'TN':TN,
                                 'Precision': precision, 
                                 'Recall': recall,
                                 'F1': f1_score}
                
        
    #COMPUTE MICRO-AVERAGED PRECISION, RECALL AND F1
    
    #Counters to track class aggregated metrics
    TP_sum, FP_sum, FN_sum = 0, 0, 0
     
    #Iterate through all the classes
    for aspect_key in class_metrics.keys():
        
        #Get the TP
        TP_sum += class_metrics[aspect_key]['TP']
        
        #Get the FP
        FP_sum += class_metrics[aspect_key]['FP']
        
        #Get the FN
        FN_sum += class_metrics[aspect_key]['FN']
        
    #Micro-precision
    micro_precision = TP_sum/(TP_sum + FP_sum)
    
    #Micro recall
    micro_recall = TP_sum/(TP_sum + FN_sum)
    
    #Micro F1
    micro_f1 = 2*((micro_precision * micro_recall)/(micro_precision + micro_recall))
    
    #Return class level metrics, micro-precision, micro-recall and micro-f1
    return class_metrics, micro_precision, micro_recall, micro_f1
    
#Run evaluation
class_metrics, micro_precision, micro_recall, micro_f1 = micro_precision_recall_f1_score(merged_df['True Aspects'],merged_df['Detected aspects'])

#### Class-level Evaluation

In [21]:
pd.DataFrame(class_metrics).T.astype({'TP': 'int32','FP': 'int32','FN': 'int32','TN': 'int32'})

,TP,FP,FN,TN,Precision,Recall,F1
price,6,0,22,349,1.000000,0.214286,0.352941
speed,15,1,18,343,0.937500,0.454545,0.612245
reliability,4,0,28,345,1.000000,0.125000,0.222222
coverage,3,1,14,359,0.750000,0.176471,0.285714
customer service,20,33,5,319,0.377358,0.800000,0.512821
trustworthiness,0,1,6,370,0.000000,0.000000,0.000000


**DISCUSSION:**
- Model does poorly retrieving all the different aspects
- When the model does predict speed and reliability, it is correct most of the time. This does not hold up for the other aspects
- As evidenced by the micro-averaged F1, the current model is not sufficient for our predictive task and could do with significant refinement or replacement altogether.

#### Global-level Evaluation

In [22]:
print(f"Micro-averaged precision: {round(micro_precision,3)}")
print(f"Micro-averaged recall: {round(micro_recall,3)}")
print(f"Micro-averaged F1 score: {round(micro_f1,3)}")

Micro-averaged precision: 0.571
Micro-averaged recall: 0.34
Micro-averaged F1 score: 0.427


There are certainly parameters that can be tweaked around/optimized (e.g. similarity score threshold, aspect category names, etc.) through a grid search, but I don't expect the improvement to be extremely significant, especially given my findings from the annotation phase (see section below). Even still, tinkering with these to improve performance doesn't make the model very robust

### Aspect Extraction Evaluation

We only consider the sentiment predicted for the aspects the model determined correctly

In [23]:
def aspect_sentiment_accuracy(true_aspects,aspect_preds,true_sentiment,sentiment_preds):
    
    #Dictionary to note the sentiment prediction accuracy for the different aspects
    aspect_accuracy = {}
    
    #Track the number of correct preds and total preds across all the classes
    global_correct_preds, global_total_preds = 0, 0
    
    #Iterate through all the aspects
    for aspect in aspects:
        
        #Initialize counters for number of correct predictions and number of total predictions
        correct_preds, total_preds = 0, 0
        
        #Iterate through all the tweets
        for idx in range(len(aspect_preds)):
            
            #If the predicted aspect is truly in the tweet
            if (aspect in aspect_preds[idx]) and (aspect in true_aspects[idx]):
                
                #Format the predicted list to a numpy array
                model_preds = np.array(aspect_preds[idx]) #Model preds
                
                true_preds = np.array(true_aspects[idx]) #True preds
                
                #Find the corresponding sentiment of the correctly predicted aspect
                #1. In model preds
                sentiment_pred = sentiment_preds[idx][np.argwhere(model_preds == aspect)[0][0]]
                
                #1. In true preds
                true_sentiment_pred = true_sentiment[idx][np.argwhere(true_preds == aspect)[0][0]]
                
                #If the predicted sentiment for the aspect is equal to the true sentiment
                if sentiment_pred == true_sentiment_pred:
                    
                    #Record as correct prediction
                    correct_preds += 1
                    global_correct_preds += 1 #Global case
                    
                
                #Record a prediction regardless of if correct or not
                total_preds += 1
                global_total_preds += 1 #Global case
                
                
        #Note down the final class-level accuracy
        try:
            aspect_accuracy[aspect] = correct_preds/total_preds
        except ZeroDivisionError:
            aspect_accuracy[aspect] = 'No correct aspect detection for this aspect'
            
                
    #Compute the global/micro accuracy (across all aspects)
    micro_accuracy = global_correct_preds/global_total_preds
    
    #Compute the macro accuracy (unweighted average from all the classes)
    macro_accuracy = np.mean([aspect_accuracy[aspect] for aspect in aspect_accuracy.keys() if isinstance(aspect_accuracy[aspect],float)])
    
    #Return class level metrics, micro-accuracy, and macro accuracy
    return aspect_accuracy, micro_accuracy, macro_accuracy
    
#Run evaluation
class_metrics, micro_accuracy, macro_accuracy = aspect_sentiment_accuracy(merged_df['True Aspects'],
                                                                          merged_df['Detected aspects'],
                                                                          merged_df['True Sentiment'],
                                                                          merged_df['Corresponding sentiment'])

In [24]:
class_metrics

{'price': 0.8333333333333334,
 'speed': 0.7333333333333333,
 'reliability': 1.0,
 'coverage': 0.6666666666666666,
 'customer service': 0.8,
 'trustworthiness': 'No correct aspect detection for this aspect'}

In [25]:
micro_accuracy

0.7916666666666666

In [26]:
macro_accuracy

0.8066666666666666

The model does pretty well determining the sentiment when the aspect is actually correctly determined. We see micro and macro accuracy scores at about 80%. We note however that the model performs poorly predicting the sentiment for coverage. It predicts perfectly for reliability.

Also we note that after switching 'cost' in the previous implementation with 'price,' the model does much better on both extraction and sentiment category prediction

**Note:** This part of the model was implemented using the ABSA package by ScalaConsultants

---

The above results obtained using the ABSA package by ScalaConsultants were obtained by passing in words that were specifically in the tweets. Based on the documentation, the ABSA package is also able to determine sentiment towards an aspect even when the aspect is not explicitly mentioned. This is explored below. Depending on the results, we might not need to focus efforts on creating an entirely new aspect category sentiment prediction pipeline. We can instead simply focus efforts on improving aspect category extraction

### ABSA using aspect categories rather than aspect terms

In [27]:
#Drop NAs in true aspect dataframe
tdf = true_aspects_df.dropna(subset=['Aspects','Sentiment']).reset_index(drop=True)

#Merge both dataframes (true dataframe and preds dataframe) on the tweet text
tdf_and_preds = pd.merge(tdf,absa_df.dropna()[['Tweets','Detected aspects']],on='Tweets',how='inner')

#Quick preview
tdf_and_preds.head()

,Tweets,Aspects,Sentiment,Detected aspects
0,spectranetng your response just proves how hor...,[customer service],[Negative],[customer service]
1,spectranetng dontusespectranet incredibly poor...,"[coverage, customer service, price]","[Negative, Negative, Negative]","[coverage, customer service]"
2,spectranetng its a disgusting customer service...,[customer service],[Negative],[customer service]
3,spectranetng hwfa why so slow,[speed],[Negative],[speed]
4,kunleeko drdammie giditraffic plentygadgets dr...,[speed],[Negative],[customer service]


In [28]:
def compute_sentiment_with_categories(tweets,detected_aspects):
    
    #List to store the tweet aspect sentiments
    tweet_aspect_sentiments = []
    
    for idx in range(len(detected_aspects)):
        
        #Calculate the sentiment for the different detected aspects
        sentiment = nlp(tweets[idx] ,aspects = detected_aspects[idx])
        
        #Add the predicted sentiment to the list
        tweet_aspect_sentiments.append([str(sentiment.subtasks[aspect].sentiment).replace('Sentiment.','').capitalize() for aspect in sentiment.aspects])
        
    
    df = pd.concat([tweets, detected_aspects], axis=1).reset_index(drop=True)
    df['Corresponding sentiment'] = tweet_aspect_sentiments
        
    return df

In [29]:
absa_cat = compute_sentiment_with_categories(tdf_and_preds.Tweets,
                                             tdf_and_preds['Detected aspects'])

In [30]:
class_metrics, micro_accuracy, macro_accuracy = aspect_sentiment_accuracy(tdf_and_preds['Aspects'],
                                                                          absa_cat['Detected aspects'],
                                                                          tdf_and_preds['Sentiment'],
                                                                          absa_cat['Corresponding sentiment'])

In [31]:
class_metrics

{'price': 0.6666666666666666,
 'speed': 0.8,
 'reliability': 1.0,
 'coverage': 0.6666666666666666,
 'customer service': 0.8947368421052632,
 'trustworthiness': 'No correct aspect detection for this aspect'}

In [32]:
micro_accuracy

0.8260869565217391

In [33]:
macro_accuracy

0.8056140350877193

ABSA model does even better when just the aspect categories are fed in! Not the aspect terms

---

# Annotating the 2nd ABSA data extraction

In [38]:
#Set to store all seen words
seen_words = set()

#Set to store all aspect implying words found – to avoid recomputing similarity scores
aspect_implying_words_glob = set()

#Dictionary categorizing all aspect-implying words into their relevant aspects
aspects_with_implying_words = {'price':set(),'speed':set(),'reliability':set(),'coverage':set(), 
                               'customer service':set(),'trustworthiness':set()}

#List to store detected aspects and their sentiments
df_list = []

#Similarity threshold
sim_thresh = 0.6

#Chunk tags to match – i.e. parts of speech to extract
CHUNK_TAG = """
MATCH: {<NN>+|<NN.*>+}
{<JJ.*>?}
{<RB.*>?}
"""

#Initialize chunk tag parser
cp = nltk.RegexpParser(CHUNK_TAG)

#Iterate through all the tweets
for tweet in new_annotations['Cleaned text']:
    
    #Set to store the detected aspects at the sentence level
    # detected_aspects = set()
    
    #Dictionary to store the sentiment value for each seen aspect
    sentence_lvl_aspect_sentiment = {'price':[],'speed':[],'reliability':[],'coverage':[], 
                                     'customer service':[], 'trustworthiness':[]}
        
    #Split the tweet into words
    text = tweet.split()

    #Tag the words with their part of speech
    tokens_tag = pos_tag(text)
    
    #Get the words with relevant POS (noun, adverbs, adjectives)
    chunk_result = cp.parse(tokens_tag)
    
    #Extract chunk results from tree into list 
    chunk_items = [list(n) for n in chunk_result if isinstance(n, nltk.tree.Tree)]
    
    #Finally fuse/extract chunked words to get (noun) phrases, nouns, adverbs, adjectives
    #1. List to store the words
    matched_words = []
    
    #2. Iterate through the chunked words lists and get the relevant words
    for item in chunk_items:
        if len(item) > 1:
            full_string = []

            for word in item:
                full_string.append(word[0])

            matched_words.append(' '.join(full_string))

        else:
            matched_words.append(item[0][0])
        
    #Iterate through all the words
    for word_in_focus in matched_words:
        
        #If the word has been seen before
        if word_in_focus in seen_words:
            
            #Check if the word is an aspect-implying word
            if word_in_focus in aspect_implying_words_glob:
                
                #List to store all the aspects found to related to the certain word/token
                aspects_implied = []
            
                #If it is an aspect-implying word, iterate through all the aspects
                for aspect in aspects_with_implying_words.keys():
                    
                    #Check if the word_in_focus was noted as a word implying the aspect
                    if word_in_focus in aspects_with_implying_words[aspect]:
                        
                        #Get all the aspects the word_in_focus implies
                        aspects_implied.append(aspect)
                        
            
            else:
                continue
                    
         
        #If the word hasn't been seen before
        else:
            
            #Mark the word as seen now
            seen_words.add(word_in_focus)
                
            #List to store all the aspects found to related to the certain word/token
            #Ideally a given word won't imply multiple of the aspects as they are fairly independent
            #-but just in case 
            aspects_implied = []

            #Iterate through all the aspects
            for aspect,asp_token in aspects_with_token:

                #Translate word_in_focus to word vector
                spacy_token = spacy_nlp(word_in_focus)

                #Compute the similarity between the two word vectors (i.e. the two words)
                #Round up to 1 d.p.
                similarity_score = round(asp_token.similarity(spacy_token),1)

                #If the max similarity score seen is greater than the threshold
                if similarity_score >= sim_thresh:

                    #Add the word to the set of all aspect-implying words seen
                    aspect_implying_words_glob.add(word_in_focus)

                    #Add the word to the dictionary of the relevant aspect word
                    aspects_with_implying_words[aspect].add(word_in_focus)

                    #Note that the aspect has been found in this particular sentence
                    # detected_aspects.add(aspect)

                    #Add the aspect to the list of aspects that the word_in_focus implies
                    aspects_implied.append(aspect)


                #If the word is not an aspect implying word, continue to next word
                else:

                    continue
                
        #Calculate the sentiment scores for the aspect_implying word in the current sentence
        sentiment = nlp(tweet ,aspects = [word_in_focus])
        sentiment_scores = sentiment.subtasks[word_in_focus].examples[0].scores

        #Note down the scores for all the implied aspects
        for aspect in aspects_implied:
            sentence_lvl_aspect_sentiment[aspect].append(sentiment_scores)
    
    #List to store the detected aspects from the sentence
    detected_aspects = []
    
    #List to store the determined sentiments of the detected aspects
    detected_sentiments = []
    
    #Iterate through all the aspects
    for aspect in sentence_lvl_aspect_sentiment.keys():
        
        #If the aspect was detected in the sentence
        if sentence_lvl_aspect_sentiment[aspect]:
            
            #Record this
            detected_aspects.append(aspect)
            
            #Calculate the average sentiment scores across the different terms
            avg_senti_score = np.array(sentence_lvl_aspect_sentiment[aspect]).mean(axis=0)
            
            #Get the sentiment category (neutral,negative,positive) with the largest probability
            max_idx = np.argmax(avg_senti_score)

            if max_idx == 2:

                detected_sentiments.append("Positive")

            elif max_idx == 1:

                detected_sentiments.append("Negative")

            else:

                detected_sentiments.append("Neutral")
    
    #Add the detected aspects and sentiments from the sentence to the list
    if detected_aspects:
        df_list.append([tweet,detected_aspects,detected_sentiments])
    else:
        df_list.append([tweet,None,None])

            

/Users/koredeakande/opt/anaconda3/envs/capstone/lib/python3.7/site-packages/ipykernel_launcher.py:111: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


In [39]:
newly_annotated_df = pd.DataFrame(df_list, 
                                  columns=['Cleaned tweets','Detected aspects','Corresponding sentiment'])
newly_annotated_df.insert(0,'Tweets',new_annotations.Text)

In [41]:
newly_annotated_df.to_csv('../data/model-generated/tweet_absa_second_annotation.csv',index=False)